In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
%load_ext tensorboard

In [2]:
'''
import glob
import pickle
files = glob.glob("../../datasets/ant-*.csv", recursive = True)
globa = pd.concat(map(pd.read_csv, files))
pickle.dump( globa, open( "ant_master.p", "wb" ) )
'''

'\nimport glob\nimport pickle\nfiles = glob.glob("../../datasets/ant-*.csv", recursive = True)\ngloba = pd.concat(map(pd.read_csv, files))\npickle.dump( globa, open( "ant_master.p", "wb" ) )\n'

In [3]:
ant_master = pickle.load( open( "xerces_master.p", "rb" ) )

In [4]:
ant_master

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,xerces,1.2.0,org.xml.sax.SAXException,6,3,3,0,10,0,0,...,1.000000,0,0.850000,0.555556,1,2,7.833333,3,1.0000,0
1,xerces,1.2.0,org.apache.xerces.parsers.DOMParser,63,2,3,27,225,793,7,...,0.896552,6,0.678947,0.152074,2,11,88.698413,6,1.6984,0
2,xerces,1.2.0,org.apache.xerces.dom.RangeExceptionImpl,1,5,0,1,2,0,1,...,0.000000,0,1.000000,1.000000,0,0,5.000000,0,0.0000,1
3,xerces,1.2.0,org.apache.html.dom.HTMLIFrameElementImpl,21,4,0,2,25,210,0,...,0.000000,0,0.824561,0.523810,0,0,4.714286,1,0.9524,0
4,xerces,1.2.0,org.apache.xerces.dom.ChildAndParentNode,25,3,3,13,67,34,3,...,0.833333,3,0.684932,0.205000,2,22,43.560000,11,2.2800,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,xerces,init,org.apache.xerces.dom.DeferredElementDefinitio...,4,3,0,5,11,0,1,...,0.500000,0,0.940000,0.500000,2,12,17.000000,2,1.0000,0
158,xerces,init,org.apache.xerces.utils.XMLMessageProvider,3,1,0,5,3,3,5,...,0.000000,0,0.000000,0.583333,0,0,0.000000,1,1.0000,0
159,xerces,init,org.xml.sax.ErrorHandler,3,1,0,0,3,3,0,...,0.000000,0,0.000000,1.000000,0,0,0.000000,1,1.0000,2
160,xerces,init,org.apache.xerces.utils.XMLMessages,5,1,0,2,14,0,1,...,0.009404,0,0.000000,0.500000,0,0,155.600000,9,2.2000,0


In [5]:
test = ant_master[ant_master['version']=='init']
test

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,xerces,init,org.xml.sax.SAXParseException,8,4,0,0,14,0,0,...,1.000000,0,0.833333,0.450000,0,0,10.750000,1,0.5000,2
1,xerces,init,org.apache.xerces.utils.UTF8DataChunk,21,1,0,6,35,68,1,...,0.727273,4,0.000000,0.271429,0,0,67.904762,24,4.7619,0
2,xerces,init,org.apache.xerces.domx.events.EventListener,1,1,0,4,1,0,3,...,0.000000,0,0.000000,1.000000,0,0,0.000000,1,1.0000,0
3,xerces,init,org.apache.xerces.parsers.DOMParser,56,2,2,20,207,778,4,...,0.961538,6,0.704918,0.183471,2,14,82.553571,14,1.7679,4
4,xerces,init,org.apache.xerces.dom.DocumentFragmentImpl,5,2,0,3,8,10,1,...,0.000000,0,0.936170,0.466667,1,4,3.800000,1,0.6000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,xerces,init,org.apache.xerces.dom.DeferredElementDefinitio...,4,3,0,5,11,0,1,...,0.500000,0,0.940000,0.500000,2,12,17.000000,2,1.0000,0
158,xerces,init,org.apache.xerces.utils.XMLMessageProvider,3,1,0,5,3,3,5,...,0.000000,0,0.000000,0.583333,0,0,0.000000,1,1.0000,0
159,xerces,init,org.xml.sax.ErrorHandler,3,1,0,0,3,3,0,...,0.000000,0,0.000000,1.000000,0,0,0.000000,1,1.0000,2
160,xerces,init,org.apache.xerces.utils.XMLMessages,5,1,0,2,14,0,1,...,0.009404,0,0.000000,0.500000,0,0,155.600000,9,2.2000,0


In [6]:
train = ant_master[ant_master['version']!='init']
train

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,xerces,1.2.0,org.xml.sax.SAXException,6,3,3,0,10,0,0,...,1.000000,0,0.850000,0.555556,1,2,7.833333,3,1.0000,0
1,xerces,1.2.0,org.apache.xerces.parsers.DOMParser,63,2,3,27,225,793,7,...,0.896552,6,0.678947,0.152074,2,11,88.698413,6,1.6984,0
2,xerces,1.2.0,org.apache.xerces.dom.RangeExceptionImpl,1,5,0,1,2,0,1,...,0.000000,0,1.000000,1.000000,0,0,5.000000,0,0.0000,1
3,xerces,1.2.0,org.apache.html.dom.HTMLIFrameElementImpl,21,4,0,2,25,210,0,...,0.000000,0,0.824561,0.523810,0,0,4.714286,1,0.9524,0
4,xerces,1.2.0,org.apache.xerces.dom.ChildAndParentNode,25,3,3,13,67,34,3,...,0.833333,3,0.684932,0.205000,2,22,43.560000,11,2.2800,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,xerces,1.3.0,org.xml.sax.ErrorHandler,3,1,0,0,3,3,0,...,0.000000,0,0.000000,1.000000,0,0,0.000000,1,1.0000,0
449,xerces,1.3.0,org.apache.xerces.readers.XMLCatalogHandler,11,1,1,1,18,15,1,...,1.000000,0,0.000000,0.431818,0,0,8.090909,1,0.9091,0
450,xerces,1.3.0,org.apache.xerces.utils.IntStack,8,1,0,2,14,0,2,...,1.000000,0,0.000000,0.625000,0,0,16.750000,4,1.3750,0
451,xerces,1.3.0,org.apache.xerces.msg.SchemaMessages,3,3,0,0,4,1,0,...,0.000000,0,0.974359,1.000000,0,0,159.000000,1,0.3333,4


In [7]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [8]:
x_train

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
0,6,3,3,0,10,0,0,0,6,0.000000,54,1.000000,0,0.850000,0.555556,1,2,7.833333,3,1.0000
1,63,2,3,27,225,793,7,20,42,0.872080,5680,0.896552,6,0.678947,0.152074,2,11,88.698413,6,1.6984
2,1,5,0,1,2,0,1,0,1,2.000000,6,0.000000,0,1.000000,1.000000,0,0,5.000000,0,0.0000
3,21,4,0,2,25,210,0,2,21,2.000000,120,0.000000,0,0.824561,0.523810,0,0,4.714286,1,0.9524
4,25,3,3,13,67,34,3,11,14,0.638889,1120,0.833333,3,0.684932,0.205000,2,22,43.560000,11,2.2800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,3,1,0,0,3,3,0,0,3,2.000000,3,0.000000,0,0.000000,1.000000,0,0,0.000000,1,1.0000
449,11,1,1,1,18,15,1,0,11,0.500000,102,1.000000,0,0.000000,0.431818,0,0,8.090909,1,0.9091
450,8,1,0,2,14,0,2,0,7,0.357143,144,1.000000,0,0.000000,0.625000,0,0,16.750000,4,1.3750
451,3,3,0,0,4,1,0,0,2,0.500000,481,0.000000,0,0.974359,1.000000,0,0,159.000000,1,0.3333


In [9]:
x_test

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
0,8,4,0,0,14,0,0,0,8,0.428571,98,1.000000,0,0.833333,0.450000,0,0,10.750000,1,0.5000
1,21,1,0,6,35,68,1,5,12,0.677273,1458,0.727273,4,0.000000,0.271429,0,0,67.904762,24,4.7619
2,1,1,0,4,1,0,3,1,1,2.000000,1,0.000000,0,0.000000,1.000000,0,0,0.000000,1,1.0000
3,56,2,2,20,207,778,4,17,37,0.856643,4705,0.961538,6,0.704918,0.183471,2,14,82.553571,14,1.7679
4,5,2,0,3,8,10,1,3,5,1.250000,25,0.000000,0,0.936170,0.466667,1,4,3.800000,1,0.6000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,4,3,0,5,11,0,1,5,1,0.666667,74,0.500000,0,0.940000,0.500000,2,12,17.000000,2,1.0000
158,3,1,0,5,3,3,5,0,3,2.000000,3,0.000000,0,0.000000,0.583333,0,0,0.000000,1,1.0000
159,3,1,0,0,3,3,0,0,3,2.000000,3,0.000000,0,0.000000,1.000000,0,0,0.000000,1,1.0000
160,5,1,0,2,14,0,1,1,4,1.243730,1102,0.009404,0,0.000000,0.500000,0,0,155.600000,9,2.2000


In [10]:
y_train = train['bug']
y_test = test['bug']

In [11]:
y_train

0      0
1      0
2      1
3      0
4      2
      ..
448    0
449    0
450    0
451    4
452    0
Name: bug, Length: 1481, dtype: int64

In [12]:
y_test

0      2
1      0
2      0
3      4
4      0
      ..
157    0
158    0
159    2
160    0
161    2
Name: bug, Length: 162, dtype: int64

In [13]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [14]:
x_train_np

array([[  6.        ,   3.        ,   3.        , ...,   7.83333333,
          3.        ,   1.        ],
       [ 63.        ,   2.        ,   3.        , ...,  88.6984127 ,
          6.        ,   1.6984    ],
       [  1.        ,   5.        ,   0.        , ...,   5.        ,
          0.        ,   0.        ],
       ...,
       [  8.        ,   1.        ,   0.        , ...,  16.75      ,
          4.        ,   1.375     ],
       [  3.        ,   3.        ,   0.        , ..., 159.        ,
          1.        ,   0.3333    ],
       [  2.        ,   3.        ,   0.        , ...,   3.5       ,
          0.        ,   0.        ]])

In [15]:
x_test_np

array([[  8.       ,   4.       ,   0.       , ...,  10.75     ,
          1.       ,   0.5      ],
       [ 21.       ,   1.       ,   0.       , ...,  67.9047619,
         24.       ,   4.7619   ],
       [  1.       ,   1.       ,   0.       , ...,   0.       ,
          1.       ,   1.       ],
       ...,
       [  3.       ,   1.       ,   0.       , ...,   0.       ,
          1.       ,   1.       ],
       [  5.       ,   1.       ,   0.       , ..., 155.6      ,
          9.       ,   2.2      ],
       [  1.       ,   1.       ,   0.       , ...,   0.       ,
          1.       ,   1.       ]])

In [16]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [17]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [18]:
train_x

array([[[  6.        ,   3.        ,   3.        , ...,   7.83333333,
           3.        ,   1.        ]],

       [[ 63.        ,   2.        ,   3.        , ...,  88.6984127 ,
           6.        ,   1.6984    ]],

       [[  1.        ,   5.        ,   0.        , ...,   5.        ,
           0.        ,   0.        ]],

       ...,

       [[  8.        ,   1.        ,   0.        , ...,  16.75      ,
           4.        ,   1.375     ]],

       [[  3.        ,   3.        ,   0.        , ..., 159.        ,
           1.        ,   0.3333    ]],

       [[  2.        ,   3.        ,   0.        , ...,   3.5       ,
           0.        ,   0.        ]]])

In [19]:
test_y

array([ 2,  0,  0,  4,  0,  1,  0,  0,  2,  2,  0,  1,  0,  2,  1,  0,  0,
        0,  0,  0,  9,  1,  3,  0,  0,  0,  1,  0,  0,  0,  0,  0,  2,  2,
        2,  0,  2,  0,  1,  0,  0,  2,  2,  0,  0,  0,  0,  2,  0,  0,  0,
        2,  0,  0,  0,  2,  2,  2,  0,  0,  0,  1,  2,  2,  1,  2,  0,  0,
        2,  0,  0,  1,  0,  1,  2,  0,  2,  2,  0,  0,  1,  0,  0,  2,  5,
        0,  3,  0,  1,  0,  0,  1,  0,  2,  0,  0,  2,  0,  0,  2,  0,  2,
        0,  0,  1,  2,  2,  2,  2,  5,  0,  0,  1,  2,  2,  0,  0,  8,  2,
        0,  0,  2,  0,  0,  0,  0,  2,  1,  2,  1,  2,  0,  1,  0,  0,  0,
        0,  0,  1,  2, 11,  0,  2,  0,  2,  1,  2,  0,  1,  0,  5,  0,  0,
        1,  0,  2,  2,  0,  0,  2,  0,  2])

In [20]:
# Designing and initializing the model.
from keras.layers import LSTM, Dense, SimpleRNN
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, dropout = 0.2, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.005) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [21]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])

Epoch 1/100
15/15 [==============================] - 8s 28ms/step - loss: 13.8812 - mse: 13.8812 - mae: 1.5456 - root_mean_squared_error: 3.7257 - mean_squared_logarithmic_error: 0.6264
Epoch 2/100
15/15 [==============================] - 0s 10ms/step - loss: 12.6677 - mse: 12.6677 - mae: 1.4829 - root_mean_squared_error: 3.5592 - mean_squared_logarithmic_error: 0.4996
Epoch 3/100
15/15 [==============================] - 0s 10ms/step - loss: 12.2854 - mse: 12.2854 - mae: 1.5726 - root_mean_squared_error: 3.5051 - mean_squared_logarithmic_error: 0.5270
Epoch 4/100
15/15 [==============================] - 0s 11ms/step - loss: 12.1535 - mse: 12.1535 - mae: 1.5638 - root_mean_squared_error: 3.4862 - mean_squared_logarithmic_error: 0.5219
Epoch 5/100
15/15 [==============================] - 0s 11ms/step - loss: 12.2023 - mse: 12.2023 - mae: 1.5941 - root_mean_squared_error: 3.4932 - mean_squared_logarithmic_error: 0.5414
Epoch 6/100
15/15 [==============================] - 0s 11ms/step - lo

15/15 [==============================] - 0s 10ms/step - loss: 11.4431 - mse: 11.4431 - mae: 1.5237 - root_mean_squared_error: 3.3828 - mean_squared_logarithmic_error: 0.4964
Epoch 88/100
15/15 [==============================] - 0s 8ms/step - loss: 11.4876 - mse: 11.4876 - mae: 1.5156 - root_mean_squared_error: 3.3893 - mean_squared_logarithmic_error: 0.4885
Epoch 89/100
15/15 [==============================] - 0s 10ms/step - loss: 11.7918 - mse: 11.7918 - mae: 1.5234 - root_mean_squared_error: 3.4339 - mean_squared_logarithmic_error: 0.4953
Epoch 90/100
15/15 [==============================] - 0s 9ms/step - loss: 12.4437 - mse: 12.4437 - mae: 1.5715 - root_mean_squared_error: 3.5276 - mean_squared_logarithmic_error: 0.5314
Epoch 91/100
15/15 [==============================] - 0s 10ms/step - loss: 12.0998 - mse: 12.0998 - mae: 1.5463 - root_mean_squared_error: 3.4785 - mean_squared_logarithmic_error: 0.5120
Epoch 92/100
15/15 [==============================] - 0s 10ms/step - loss: 12.52

In [22]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

In [23]:
score = model.evaluate(test_x, test_y)
score

6/6 [==============================] - 2s 5ms/step - loss: 3.8230 - mse: 3.8230 - mae: 1.5836 - root_mean_squared_error: 1.9552 - mean_squared_logarithmic_error: 0.7026


[3.8229832649230957,
 3.8229832649230957,
 1.5835866928100586,
 1.955245018005371,
 0.7026405930519104]